In [20]:
#Importing Libraries and Loading Dataset
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
#Retrive date from csv file
file_path='/content/customer_support_tickets.csv'
df=pd.read_csv(file_path)
#Exploring the dataset
print("Shape (rows,columns):", df.shape)
print('\nInfo:')
print(df.info())
print("\nMissing Values:")
print(df.isna().sum())
df.head()

Shape (rows,columns): (8469, 17)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority       

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [21]:
import re
MESSAGE_COL ='Ticket Description' #this is the main text column to store messages
def clean_message(text):
  if pd.isna(text):
    return ""
  text=text.lower()  #converting it to lowercase
  text=re.sub(r'[^a-z0-9\s]', ' ', text)  #removing the special characters
  text=re.sub(r'\s+', ' ', text).strip() #removing the extra spaces
  return text
df['clean_message']=df[MESSAGE_COL].astype(str).apply(clean_message)
df[['Ticket Description', 'clean_message']].head()

,Ticket Description,clean_message
0,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...
1,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...
2,I'm facing a problem with my {product_purchase...,i m facing a problem with my product purchased...
3,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...
4,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...


In [22]:
def classify_issue(text):
  #For Payment Issue
  if any(word in text for word in[
      'payment', 'card', 'credit', 'debit', 'billing', 'charged', 'transaction', 'invoice'
  ]):
    return 'PAYMENT'
  #For Login Issue
  if any(word in text for word in[
      'login', 'log in', 'sign in', 'password', 'otp', '2fa', 'account', 'locked', 'reset password'
  ]):
    return 'LOGIN'
  #For Delivery Issue
  if any(word in text for word in[
      'delivery', 'deliver', 'shipping', 'shipment', 'courier', 'tracking', 'track', 'order not arrived', 'delayed', 'delay'
  ]):
    return 'DELIVERY'
  #For Refund Issue
  if any(word in text for word in[
      'refund', 'money back', 'return', 'cancel order',  'chargeback'
  ]):
    return 'REFUND'
  #For Bug Issue
  if any(word in text for word in [
        'bug', 'error', 'crash', 'not working', 'issue', 'fail', 'failed', 'glitch', 'problem'
    ]):
        return 'BUG'
  return 'GENERAL' #For General Issues
df['issue_type']=df['clean_message'].apply(classify_issue)
df[['Ticket ID', 'Ticket Description', 'clean_message', 'issue_type']].head(10)

,Ticket ID,Ticket Description,clean_message,issue_type
0,1,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...,PAYMENT
1,2,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...,BUG
2,3,I'm facing a problem with my {product_purchase...,i m facing a problem with my product purchased...,BUG
3,4,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...,BUG
4,5,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...,DELIVERY
5,6,I'm facing a problem with my {product_purchase...,i m facing a problem with my product purchased...,BUG
6,7,I'm unable to access my {product_purchased} ac...,i m unable to access my product purchased acco...,LOGIN
7,8,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...,BUG
8,9,I'm having an issue with the {product_purchase...,i m having an issue with the product purchased...,BUG
9,10,My {product_purchased} is making strange noise...,my product purchased is making strange noises ...,BUG


In [23]:
def assign_priority(text):
    if any(word in text for word in [
        'urgent', 'immediately', 'asap', 'right now','system down', 'cannot login', "can't login", 'unable to access', 'payment failed', 'card declined', 'unauthorized', 'hacked'
    ]):
        return 'P0' #Critical issue
    if any(word in text for word in [
        'error', 'failed', 'not working', 'does not work', 'cannot', "can't", 'blocked', 'account locked', 'crash', 'not opening'
    ]):
        return 'P1' #High Priority Issue
    if any(word in text for word in [
        'slow', 'delay', 'delayed', 'late', 'not received', 'no update', 'taking too long'
    ]):
        return 'P2' #Moderate Priority Issue
    return 'P3' #Low Priority Issue
df['auto_priority'] = df['clean_message'].apply(assign_priority)
df[['Ticket ID', 'clean_message', 'issue_type', 'auto_priority']].head(10)

,Ticket ID,clean_message,issue_type,auto_priority
0,1,i m having an issue with the product purchased...,PAYMENT,P3
1,2,i m having an issue with the product purchased...,BUG,P3
2,3,i m facing a problem with my product purchased...,BUG,P3
3,4,i m having an issue with the product purchased...,BUG,P3
4,5,i m having an issue with the product purchased...,DELIVERY,P3
5,6,i m facing a problem with my product purchased...,BUG,P3
6,7,i m unable to access my product purchased acco...,LOGIN,P0
7,8,i m having an issue with the product purchased...,BUG,P3
8,9,i m having an issue with the product purchased...,BUG,P2
9,10,my product purchased is making strange noises ...,BUG,P3


In [24]:
sla_mapping = {
    'P0': 2, #2 Hrs
    'P1': 4, #4 Hrs
    'P2': 24, #24 Hrs (1 day)
    'P3': 72  #72 Hrs (3 days)
}
df['sla_hours'] = df['auto_priority'].map(sla_mapping)
#Converting the date of purchase to date time
df['created_at'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')
#Calculating the due time of the process
df['due_time'] = df['created_at'] + pd.to_timedelta(df['sla_hours'], unit='h')
df[['Ticket ID', 'created_at', 'auto_priority', 'sla_hours', 'due_time']].head(10)

,Ticket ID,created_at,auto_priority,sla_hours,due_time
0,1,2021-03-22,P3,72,2021-03-25 00:00:00
1,2,2021-05-22,P3,72,2021-05-25 00:00:00
2,3,2020-07-14,P3,72,2020-07-17 00:00:00
3,4,2020-11-13,P3,72,2020-11-16 00:00:00
4,5,2020-02-04,P3,72,2020-02-07 00:00:00
5,6,2020-07-28,P3,72,2020-07-31 00:00:00
6,7,2020-02-23,P0,2,2020-02-23 02:00:00
7,8,2020-08-09,P3,72,2020-08-12 00:00:00
8,9,2020-07-16,P2,24,2020-07-17 00:00:00
9,10,2020-03-06,P3,72,2020-03-09 00:00:00


In [25]:
issue_counts = df['issue_type'].value_counts()
print("Tickets by issue type:\n", issue_counts, "\n")
priority_counts = df['auto_priority'].value_counts()
print("Tickets by auto_priority:\n", priority_counts, "\n")
report = df.pivot_table(
    index='issue_type',
    columns='auto_priority',
    values='Ticket ID',
    aggfunc='count',
    fill_value=0
)
print("Support Manager Report (Issue Type vs Auto Priority):")
print(report)

Tickets by issue type:
 issue_type
BUG         7003
LOGIN        741
PAYMENT      312
REFUND       227
DELIVERY     149
GENERAL       37
Name: count, dtype: int64 

Tickets by auto_priority:
 auto_priority
P3    5793
P1    1375
P2     888
P0     413
Name: count, dtype: int64 

Support Manager Report (Issue Type vs Auto Priority):
auto_priority   P0    P1   P2    P3
issue_type                         
BUG            158  1064  772  5009
DELIVERY         2    21   22   104
GENERAL         37     0    0     0
LOGIN          202   220   40   279
PAYMENT          6    43   32   231
REFUND           8    27   22   170


In [26]:
df.to_csv('triaged_tickets.csv', index=False) #Full dataset
report.to_csv('support_manager_report.csv') #Manager report